## Deliverable 2. Create a Customer Travel Destinations Map.

In [46]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [47]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Guerrero Negro,MX,27.9769,-114.0611,70.30,76,1,14.16,clear sky
1,1,San Onofre,CO,9.7359,-75.5263,77.45,92,100,3.18,overcast clouds
2,2,Natal,BR,-5.7950,-35.2094,79.02,83,20,8.05,few clouds
3,3,Hithadhoo,MV,-0.6000,73.0833,83.48,76,90,14.47,overcast clouds
4,4,Okhotsk,RU,59.3833,143.3000,34.75,65,61,14.94,broken clouds


In [48]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like to have during your trip?"))
max_temp = float(input("What is the maximum temperature you would like to have during your trip?"))

What is the minimum temperature you would like to have during your trip?75
What is the maximum temperature you would like to have during your trip?90


In [49]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

In [50]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [51]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [52]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(how = 'all') 
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [53]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,San Onofre,CO,77.45,overcast clouds,9.7359,-75.5263,
2,Natal,BR,79.02,few clouds,-5.7950,-35.2094,
3,Hithadhoo,MV,83.48,overcast clouds,-0.6000,73.0833,
5,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,
6,Keshod,IN,88.68,broken clouds,21.3000,70.2500,
7,Mahebourg,MU,77.29,broken clouds,-20.4081,57.7000,
8,Atuona,PF,78.21,clear sky,-9.8000,-139.0333,
13,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,
18,Katsuura,JP,77.14,broken clouds,35.1333,140.3000,
20,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,


In [54]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [55]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
hotel_df.dropna(subset=['Hotel Name'], inplace=True)


In [56]:
#Display clean Hotel data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,San Onofre,CO,77.45,overcast clouds,9.7359,-75.5263,Hotel Sinai
2,Natal,BR,79.02,few clouds,-5.7950,-35.2094,Villa Park Hotel | Natal
3,Hithadhoo,MV,83.48,overcast clouds,-0.6000,73.0833,Scoop Guest House
5,Saint-Pierre,RE,76.68,clear sky,-21.3393,55.4781,Lindsey Hôtel
6,Keshod,IN,88.68,broken clouds,21.3000,70.2500,Hotel New Rajdhani
7,Mahebourg,MU,77.29,broken clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
8,Atuona,PF,78.21,clear sky,-9.8000,-139.0333,Villa Enata
13,Vaini,TO,77.16,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
18,Katsuura,JP,77.14,broken clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
20,Vila Velha,BR,75.15,overcast clouds,-20.3297,-40.2925,Quality Suites Vila Velha


In [57]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [58]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [59]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))